In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/adm-lt-2024-2025-hackathon-rag/corpus.csv
/kaggle/input/adm-lt-2024-2025-hackathon-rag/example_submission.csv
/kaggle/input/adm-lt-2024-2025-hackathon-rag/train.csv
/kaggle/input/adm-lt-2024-2025-hackathon-rag/test.csv


In [2]:
# Import libraries
import pandas as pd
import numpy as np

# Load datasets
corpus_df = pd.read_csv("/kaggle/input/adm-lt-2024-2025-hackathon-rag/corpus.csv")
train_df = pd.read_csv("/kaggle/input/adm-lt-2024-2025-hackathon-rag/train.csv")
test_df = pd.read_csv("/kaggle/input/adm-lt-2024-2025-hackathon-rag/test.csv")
example_submission_df = pd.read_csv("/kaggle/input/adm-lt-2024-2025-hackathon-rag/example_submission.csv")

# Preview datasets
print("Corpus:", corpus_df.shape)
print(corpus_df.head(2), "\n")

print("Train:", train_df.shape)
print(train_df.head(2), "\n")

print("Test:", test_df.shape)
print(test_df.head(2), "\n")

print("Submission Format:")
print(example_submission_df.head())

Corpus: (40181, 2)
                                             passage     id
0  New data on viruses isolated from patients wit...   9797
1  We describe an improved method for detecting d...  11906 

Train: (3775, 4)
                                            question  \
0  Name the algorithms for counting multi-mapping...   
1  Which R/bioconductor package is used for integ...   

                                              answer    id  \
0  RNA-Seq is currently used routinely, and it pr...  3009   
1  Sushi.R is a flexible, quantitative and integr...   331   

  relevant_passage_ids  
0  [29444201 28915787]  
1           [24903420]   

Test: (1180, 2)
     id                                  question
0  1702  PBT2 has been tested for which disorder?
1  3135          What does Prevnar 13 consist of? 

Submission Format:
     id             answer
0  1702  this is an answer
1  3135  this is an answer
2   650  this is an answer
3   741  this is an answer
4  2949  this is an answer


# Stage 1: Retriever (Dense Embeddings + FAISS)


http://huggingface.co/pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb

sentence-transformers model based on BioBERT, trained on datasets relevant for natural language inference, semantic similarity, and biomedical QA

MNLI, SNLI: Natural Language Inference

SCINLI, SCITAIL, MEDNLI: Science and Medical entailment datasets

STSB: Sentence-level similarity benchmark

It outputs 768-dimensional embeddings that capture sentence meaning, ideal for semantic search in biomedical texts.



In [3]:
# ---------------------------
# Step 1: Load biomedical passages
# ---------------------------

corpus_texts = corpus_df["passage"].tolist() # actual biomedical text
corpus_ids = corpus_df["id"].tolist() # unique IDs (idk if useful)
# our retrievable "knowledge"

In [8]:
!pip install -q torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 \
  transformers sentence-transformers faiss-cpu

# Run once per session

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.7 MB/s eta 0:00:00:00:0100:01


In [5]:
import torch
print(torch.cuda.is_available())  
# We use GPU to improve performance (timely speaking)

True


In [6]:
from transformers import AutoTokenizer, AutoModel
import torch

# ---------------------------
# Step 2: Load BioBERT model
# ---------------------------

model_name = 'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb' # It maps each sentence to a dense vector
tokenizer = AutoTokenizer.from_pretrained(model_name) # convert sentences into token IDs
model = AutoModel.from_pretrained(model_name) # BioBERT model fine-tuned for semantic similarity tasks
model.eval() # turns off dropout etc. for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Get token-level embeddings (This is BioBERT outputs)

tokenizer_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

2025-05-30 14:29:42.519070: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748615382.722386      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748615382.781213      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [20]:
import gc
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from sentence_transformers import SentenceTransformer

# === SETTINGS ===
EMBEDDING_MODEL_NAME = "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
CHUNK_SIZE = 512
CHUNK_OVERLAP = 50

# === CLEAN GPU MEMORY ===
def clear_cuda():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

clear_cuda()

# === LOAD TOKENIZER & MODEL ===
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
model = SentenceTransformer(EMBEDDING_MODEL_NAME, device="cuda")

# === LANGCHAIN SPLITTER ===
splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    add_start_index=True,
    strip_whitespace=True,
)

# === CONVERT corpus_df TO LangchainDocuments ===
documents = []
for _, row in tqdm(corpus_df.iterrows(), total=len(corpus_df), desc="Preparing docs"):
    doc = LangchainDocument(
        page_content=row["passage"],
        metadata={"source": row["id"]}  # you can extend this if needed
    )
    documents.append(doc)

# === SPLIT INTO CHUNKS ===
docs_processed = []
for doc in tqdm(documents, desc="Splitting into chunks"):
    docs_processed.extend(splitter.split_documents([doc]))

# === REMOVE DUPLICATES ===
unique_docs = list({doc.page_content: doc for doc in docs_processed}.values())

# === CONVERT BACK TO DataFrame ===
chunked_df = pd.DataFrame([
    {
        "chunk": doc.page_content,
        "source_id": doc.metadata["source"],
        "start_index": doc.metadata.get("start_index", -1)
    }
    for doc in unique_docs
])

print("Chunking complete.")
print("Total unique chunks:", len(chunked_df))
print(chunked_df.head())

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Splitting into chunks: 100%|██████████| 40181/40181 [03:00<00:00, 222.53it/s]


Chunking complete.
Total unique chunks: 52339
                                               chunk  source_id  start_index
0  New data on viruses isolated from patients wit...       9797            0
1  We describe an improved method for detecting d...      11906            0
2  We have studied the effects of curare on respo...      16083            0
3  Kinetic and electrophoretic properties of 230-...      23188            0
4  Male Wistar specific-pathogen-free rats aged 2...      23469            0


In [23]:
!pip install langchain_community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: lan

In [24]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Inizializza modello BioBERT per embedding
embedding_model = HuggingFaceEmbeddings(
    model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # cosine similarity
)

# Estrai solo i testi dai Documenti
texts = [doc.page_content for doc in docs_processed]

# Calcolo embeddings con barra di progresso
print(" Calcolo degli embeddings per ogni documento...")

embeddings = []

for text in tqdm(texts, desc="Embedding passages", ncols=100):
    embedding = embedding_model.embed_query(text)
    embeddings.append(embedding)

print("Embeddings completati:", len(embeddings))



/tmp/ipykernel_35/1820959558.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


 Calcolo degli embeddings per ogni documento...


Embedding passages: 100%|█████████████████████████████████████| 52345/52345 [10:18<00:00, 84.64it/s]

Embeddings completati: 52345
 Creazione del FAISS index...


NameError: name 'FAISS' is not defined

I couldn't upload Faiss for GPU used, since here we have only CPU version that was too slow

Apparently the issue is regarding the versions, Python and CUDA doesn't match and in Kaggle you can't even change the environment. Switching into colab we managed to create a colab environment where the installation worked, but then a wrong version of numpy 

In [30]:
from cuml.neighbors import NearestNeighbors
import cupy as cp

# embeddings is a numpy array, convert in cupy
X = cp.asarray(embeddings)

nn = NearestNeighbors(n_neighbors=10, metric='cosine')
nn.fit(X)

distances, indices = nn.kneighbors(X[:5])

In [45]:
indices = indices.get()  # convert from cupy to numpy
# Get top-k passages for the first item in X[:5]
top_k_texts = [texts[int(i)] for i in indices[0]]
all_top_k_texts = [[texts[int(i)] for i in row] for row in indices]


In [46]:
question = "What disorder was the patient diagnosed with?"

answers = []
for passage in top_k_texts:
    input_qa = qa_tokenizer.encode_plus(question, passage, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = qa_model(**input_qa)
    start = torch.argmax(outputs.start_logits)
    end = torch.argmax(outputs.end_logits)
    answer = qa_tokenizer.decode(input_qa["input_ids"][0][start:end+1])
    answers.append(answer)


In [48]:
# Simple heuristic: longest answer
final_answer = max(answers, key=len)
print("🧠 Final Answer:", final_answer)
# 

🧠 Final Answer: depressive pseudodementia


In [57]:
# === RETRIEVAL FUNCTION ===
def retrieve_top_k(question, k=5):
    # Ottieni embedding normalizzato per la domanda
    q_embed = embedding_model.embed_query(question)
    q_embed_gpu = cp.asarray([q_embed])  # shape: (1, 768)

    # Trova i vicini più vicini tra i documenti indicizzati
    distances, indices = nn.kneighbors(q_embed_gpu, n_neighbors=k)
    indices = indices.get()  # da cupy a numpy

    # Ritorna i testi dei documenti top-k
    top_k_chunks = [texts[int(i)] for i in indices[0]]
    return top_k_chunks


In [58]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

qa_model_name = "ktrapeznikov/biobert_v1.1_pubmed_squad_v2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name).to("cuda")
qa_model.eval()


Some weights of the model checkpoint at ktrapeznikov/biobert_v1.1_pubmed_squad_v2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [59]:
import torch

def answer_question(question, contexts, model, tokenizer, device="cuda"):
    best_answer = ""
    best_score = float("-inf")

    for context in contexts:
        inputs = tokenizer(
            question, context, return_tensors="pt", padding=True, truncation=True, max_length=512
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

        # Prendi le migliori posizioni inizio/fine
        start_idx = torch.argmax(start_logits)
        end_idx = torch.argmax(end_logits)

        if start_idx <= end_idx:
            score = start_logits[0][start_idx] + end_logits[0][end_idx]
            answer = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx+1])
            )

            if score > best_score and answer.strip():
                best_score = score
                best_answer = answer.strip()

    return best_answer


In [60]:
question = "What is the most likely diagnosis for a patient with memory loss and depression?"
top_chunks = retrieve_top_k(question, k=10)
final_answer = answer_question(question, top_chunks, qa_model, qa_tokenizer)

print("🧠 Final Answer:", final_answer)


🧠 Final Answer: [CLS]


In [61]:
submission = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Generating answers"):
    qid = row["id"]
    question = row["question"]
    top_chunks = retrieve_top_k(question, k=10)
    answer = answer_question(question, top_chunks, qa_model, qa_tokenizer)
    submission.append((qid, answer))

# Salva il file CSV finale
submission_df = pd.DataFrame(submission, columns=["id", "answer"])
submission_df.to_csv("submission.csv", index=False)


Generating answers: 100%|██████████| 1180/1180 [04:51<00:00,  4.04it/s]


In [63]:
import pandas as pd

print("=== Submission Data Summary ===")
print(f"Total number of answers: {len(submission_df)}")

# Basic info
print("\nInfo:")
print(submission_df.info())

# Check for empty or very short answers
submission_df['answer_length'] = submission_df['answer'].apply(lambda x: len(str(x).split()))

print("\nAnswer length statistics:")
print(submission_df['answer_length'].describe())

empty_answers = submission_df[submission_df['answer_length'] <= 1]
print(f"\nNumber of empty or very short answers (<=1 token): {len(empty_answers)}")

print("\nExamples of empty or very short answers:")
print(empty_answers.head(10))

# Most common answers 
print("\nTop 10 most common answers:")
print(submission_df['answer'].value_counts().head(10))

# answer uniqueness
unique_count = submission_df['answer'].nunique()
print(f"\nNumber of unique answers: {unique_count}")

submission_df[['id', 'answer', 'answer_length']].to_csv("submission_analysis.csv", index=False)
print("\nSaved submission analysis to submission_analysis.csv")


=== Submission Data Summary ===
Total number of answers: 1180

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1180 entries, 0 to 1179
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1180 non-null   int64 
 1   answer  1180 non-null   object
dtypes: int64(1), object(1)
memory usage: 18.6+ KB
None

Answer length statistics:
count    1180.000000
mean        3.796610
std        10.471383
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max       220.000000
Name: answer_length, dtype: float64

Number of empty or very short answers (<=1 token): 752

Examples of empty or very short answers:
      id  answer  answer_length
0   1702   [CLS]              1
1   3135   [CLS]              1
2    650   [CLS]              1
3    741   [CLS]              1
4   2949   [CLS]              1
5    316   [CLS]              1
7    386   [CLS]              1
10  3851   [CLS]              1
11  1119

The number of placeholders here is bigger than I thought, apparenty FAISS is the best performing method, but this way we manage to answer some questions that FAISS was not able to